## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import sys
sys.path.append('../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,57.04,74,23,11.65,few clouds
1,1,Nikolskoye,RU,59.7035,30.7861,78.69,33,88,1.19,overcast clouds
2,2,Ongandjera,NaN,-17.8833,15.0667,76.42,16,0,9.62,clear sky
3,3,Iqaluit,CA,63.7506,-68.5145,31.69,89,99,8.01,overcast clouds
4,4,Mwense,ZM,-10.3845,28.6980,77.65,45,100,7.52,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Nikolskoye,RU,59.7035,30.7861,78.69,33,88,1.19,overcast clouds
2,2,Ongandjera,NaN,-17.8833,15.0667,76.42,16,0,9.62,clear sky
4,4,Mwense,ZM,-10.3845,28.6980,77.65,45,100,7.52,overcast clouds
6,6,Albany,US,42.6001,-73.9662,79.75,73,9,8.52,clear sky
7,7,Georgetown,MY,5.4112,100.3354,80.89,89,20,2.77,moderate rain
...,...,...,...,...,...,...,...,...,...,...
680,680,Sayyan,YE,15.1718,44.3244,77.18,10,7,20.11,clear sky
681,681,Amapa,BR,1.0000,-52.0000,78.48,87,98,3.62,overcast clouds
682,682,Natal,BR,-5.7950,-35.2094,84.85,65,40,13.67,scattered clouds
683,683,Williamsport,US,41.2412,-77.0011,81.77,64,1,3.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
# Count missing values in rows
preferred_cities_df.isnull().sum().sum()

3

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# Check if empty rows were dropped
# print (len(preferred_cities_df))
# print (len(clean_preferred_cities_df))

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nikolskoye,RU,78.69,overcast clouds,59.7035,30.7861,
4,Mwense,ZM,77.65,overcast clouds,-10.3845,28.6980,
6,Albany,US,79.75,clear sky,42.6001,-73.9662,
7,Georgetown,MY,80.89,moderate rain,5.4112,100.3354,
9,Puerto Escondido,MX,83.44,overcast clouds,15.8500,-97.0667,
12,Souillac,MU,75.36,scattered clouds,-20.5167,57.5167,
15,Mahebourg,MU,75.25,few clouds,-20.4081,57.7000,
16,Bethel,US,81.88,scattered clouds,41.3712,-73.4140,
20,Quang Ngai,VN,76.93,overcast clouds,15.1167,108.8000,
24,Toucheng,TW,80.15,light rain,24.8592,121.8233,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    #Setup Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
#Check for empty rows
hotel_df.replace("", np.NaN , inplace=True)

In [11]:
clean_hotel_df = hotel_df.dropna()

In [12]:
# Check if empty rows were dropped
print (len(hotel_df))
print (len(clean_hotel_df))

265
245


In [13]:
# Check on hotels dataframe to make sure it is correct
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nikolskoye,RU,78.69,overcast clouds,59.7035,30.7861,Tourist House - Sablino
4,Mwense,ZM,77.65,overcast clouds,-10.3845,28.6980,Mwense Council Guest House
7,Georgetown,MY,80.89,moderate rain,5.4112,100.3354,Cititel Penang
9,Puerto Escondido,MX,83.44,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
12,Souillac,MU,75.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))